In [1]:
!wget -O click-checkboxes-soft_1026085418.json https://raw.githubusercontent.com/stanfordnlp/miniwob-plusplus-demos/master/clean-demos/click-checkboxes-soft/click-checkboxes-soft_1026085418.json
!wget -O click-checkboxes-soft_1026085705.json https://raw.githubusercontent.com/stanfordnlp/miniwob-plusplus-demos/master/clean-demos/click-checkboxes-soft/click-checkboxes-soft_1026085705.json

--2023-02-14 16:47:32--  https://raw.githubusercontent.com/stanfordnlp/miniwob-plusplus-demos/master/clean-demos/click-checkboxes-soft/click-checkboxes-soft_1026085418.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106851 (104K) [text/plain]
Saving to: ‘click-checkboxes-soft_1026085418.json’

click-checkboxes-so 100%[===================>] 104.35K  --.-KB/s    in 0.003s  

2023-02-14 16:47:32 (31.7 MB/s) - ‘click-checkboxes-soft_1026085418.json’ saved [106851/106851]

--2023-02-14 16:47:32--  https://raw.githubusercontent.com/stanfordnlp/miniwob-plusplus-demos/master/clean-demos/click-checkboxes-soft/click-checkboxes-soft_1026085705.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133,

In [2]:
# read it in to inspect it
with open('click-checkboxes-soft_1026085418.json', 'r', encoding='utf-8') as f:
  soft_1026085418 = f.read()
with open('click-checkboxes-soft_1026085705.json', 'r', encoding='utf-8') as f:
  soft_1026085705 = f.read()


In [3]:
print("length of dataset in characters soft_1026085418:", len(soft_1026085418))
print("length of dataset in characters soft_1026085705:", len(soft_1026085705))

length of dataset in characters soft_1026085418: 106851
length of dataset in characters soft_1026085705: 158658


We seem to need a different vocab size for each input field below. Maybe we don't need to care about style properties yet, but to take into accounts the children and express each element with their own text descriptive values. Put together, these will represent the html code. Basically, we want to express HTML code as a text description through encodings to describe them, and output the desired action to perform context wise.

In the paper, they introduced those tuples as 

In [4]:
print(soft_1026085418[:100])
print(soft_1026085705[:100])

{"states": [{"action": null, "dom": {"bgColor": "rgb(85, 85, 85)", "top": 0, "height": 210, "width":
{"states": [{"action": null, "dom": {"bgColor": "rgb(85, 85, 85)", "top": 0, "height": 210, "width":


In [115]:
import json
#data = json.loads(soft_1026085418)
#data_soft_1026085705 = json.loads(soft_1026085705)
data = json.loads(soft_1026085705)

In [6]:
def display_main_fields(data):
  print(f'keys: {data.keys()}')
  print(len(data))
  print('taskName: ' + data['taskName'])
  print('reward: ' + str(data['reward']))
  print('utterance: ' + data['utterance'])
  print('rawReward: ' + str(data['rawReward']))
  print('amount of states:' + str(len(data['states'])))
  print('---------------------------------', end='\n')

display_main_fields(data)
display_main_fields(data_soft_1026085705)

keys: dict_keys(['states', 'taskName', 'reward', 'utterance', 'rawReward'])
5
taskName: click-checkboxes-soft
reward: 1
utterance: Select words similar to bunny, kill, despise and click Submit.
rawReward: 1
amount of states:25
---------------------------------
keys: dict_keys(['states', 'taskName', 'reward', 'utterance', 'rawReward'])
5
taskName: click-checkboxes-soft
reward: 1
utterance: Select words similar to retain, tragic, despise, bunny, reply and click Submit.
rawReward: 1
amount of states:37
---------------------------------


# Dealing with children elements

In the current json representation of html elements, the children are attached as a string. Perhaps there is no way to parsing this and we can keep it as a raw string after embedding. They also contain different properties, so the question is whether the JSON representation is sufficient to provide context to the transformer model.

In [116]:
# Recursion to check properties of a child element
# - focus
def check_children(index, children, action, html):
  #print(f'{index} - {children}')
  action_performed = None

  tag, element = recreate_html(children)
  # pretty:
  #html += "    "*index + element + "\n"
  html += element
  if 'children' in children:
    #print('going deeper')
    # Branch splits happens here. Todo: potentially split snippets at this level
    for child in children['children']:
      html, action_received = check_children(index+1, child, action, html)
      if action_received is not None:
        action_performed = action_received
  
  if 'focused' in children:
    ref = children['ref']
    if action is None:
      action_type = "none"
    else:
      action_type = action['type']

    #print(f'yes focused at index:{index} for ref:{ref} -> {children}')
    action_performed = {
        'action': action_type,
        'ref': ref
    }
    #print(f'action_performed: {action_performed}')

  # closing tag
  if tag == "INPUT_checkbox":
    html += "</input>"
  else:
    html += "</" + tag + ">"

  # closing tag pretty. todo: add matching correct tag like input type=checkbox
  #html += "    "*index + "</" + tag + ">\n"
  return html, action_performed


# fields we can drop when re-creating the HTML element
drop_fields = ['children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom']

# Recreate HTML elements
def recreate_html(child):
  fields = child.copy()
  for f in drop_fields:
    if f in fields:
      del fields[f]
  
  tag = fields.pop('tag')
  # parse input type checkbox
  if tag == 'INPUT_checkbox':
    element = '<input type="checkbox" '
  else:
    element = '<' + tag + " "
  for k, value in fields.items():
    #print(k)
    if value != '':
      element += k + '="' + str(value) + '" '
  element = element[:-1] + ">"

  #print(f'element: {element}')
  return tag, element


In [117]:
utterance = data['utterance']
print(f'utterance: {utterance}')
state = data['states']
#print(state)
print(len(state))

actions = [] # list of the actions done at each state
htmls = [] # list of different html so we can monitor changes in the episode. They should be the same but we never know

for s in state:

  action = s['action']
  dom = s['dom']
  time = s['time']


  # loop to get over all children elements
  html, action_performed = check_children(0, dom, action, "")
  htmls.append(html)
  actions.append(action_performed)

  #print(f'final html: {html}')
  #print(f'action performed: {action_performed}')
  #print('------------------------------')
    
  

  #break

if([htmls[0]]*len(htmls) == htmls):
    print("All HTMLs in data are the same")
else:
    print("All HTMLs in data are NOT the same")

print(f'list of performed actions: {actions}')

utterance: Select words similar to retain, tragic, despise, bunny, reply and click Submit.
37
All HTMLs in data are NOT the same
list of performed actions: [{'action': 'none', 'ref': 1}, {'action': 'mousedown', 'ref': 1}, {'action': 'mousedown', 'ref': 1}, {'action': 'mouseup', 'ref': 12}, {'action': 'mouseup', 'ref': 12}, {'action': 'click', 'ref': 12}, {'action': 'click', 'ref': 12}, {'action': 'mousedown', 'ref': 12}, {'action': 'mousedown', 'ref': 12}, {'action': 'mouseup', 'ref': 16}, {'action': 'mouseup', 'ref': 16}, {'action': 'click', 'ref': 16}, {'action': 'click', 'ref': 16}, {'action': 'mousedown', 'ref': 16}, {'action': 'mousedown', 'ref': 16}, {'action': 'mouseup', 'ref': 6}, {'action': 'mouseup', 'ref': 6}, {'action': 'click', 'ref': 6}, {'action': 'click', 'ref': 6}, {'action': 'mousedown', 'ref': 6}, {'action': 'mousedown', 'ref': 6}, {'action': 'mouseup', 'ref': 10}, {'action': 'mouseup', 'ref': 10}, {'action': 'click', 'ref': 10}, {'action': 'click', 'ref': 10}, {'act

In [118]:
print(len(htmls))
print(len(actions))

37
37


In [119]:
# create the input data by aggregating the parsed data
# Each html content is map to the action to be performed in order to arrive to the next state
def generate_training_data_state(utterance, htmls, actions):
  xs = []
  ys = []

  # initial step:
  xs.append(utterance + ' ' + html)
  ys.append([actions[1]['action'], actions[1]['ref']])

  i = 1;
  # last element is a terminal state, don't include it.
  while i < len(htmls)-1:
    s_actions = actions[1:i+1]
    s_actions.reverse()
    s_action = ''.join(str(x) for x in s_actions).replace("'", "")

    x = s_action + ' ' + utterance + ' ' + htmls[i]
    xs.append(x)

    action = actions[i+1]['action']
    ref = actions[i+1]['ref']
    ys.append([action, ref])

    i += 1


  return xs, ys

In [120]:
xs, ys = generate_training_data_state(utterance, htmls, actions)
actions, xs, ys

([{'action': 'none', 'ref': 1},
  {'action': 'mousedown', 'ref': 1},
  {'action': 'mousedown', 'ref': 1},
  {'action': 'mouseup', 'ref': 12},
  {'action': 'mouseup', 'ref': 12},
  {'action': 'click', 'ref': 12},
  {'action': 'click', 'ref': 12},
  {'action': 'mousedown', 'ref': 12},
  {'action': 'mousedown', 'ref': 12},
  {'action': 'mouseup', 'ref': 16},
  {'action': 'mouseup', 'ref': 16},
  {'action': 'click', 'ref': 16},
  {'action': 'click', 'ref': 16},
  {'action': 'mousedown', 'ref': 16},
  {'action': 'mousedown', 'ref': 16},
  {'action': 'mouseup', 'ref': 6},
  {'action': 'mouseup', 'ref': 6},
  {'action': 'click', 'ref': 6},
  {'action': 'click', 'ref': 6},
  {'action': 'mousedown', 'ref': 6},
  {'action': 'mousedown', 'ref': 6},
  {'action': 'mouseup', 'ref': 10},
  {'action': 'mouseup', 'ref': 10},
  {'action': 'click', 'ref': 10},
  {'action': 'click', 'ref': 10},
  {'action': 'mousedown', 'ref': 10},
  {'action': 'mousedown', 'ref': 10},
  {'action': 'mouseup', 'ref': 8},
 